In [71]:
!pip install --upgrade --no-cache-dir streamlit pyngrok pandas openpyxl groq


In [ ]:
# api keys
import os

import os
# Set this in your environment, not in code
# os.environ["GROQ_API_KEY"] = "your_api_key_here"



GROQ_API_KEY set? True
NGROK_AUTH_TOKEN set? True


In [79]:
%%writefile app.py

import streamlit as st
import pandas as pd
from groq import Groq
import os

# ----------------------------
# SETUP GROQ API
# ----------------------------
api_key = os.getenv("GROQ_API_KEY")
if not api_key:
    st.error("⚠ Please set your GROQ_API_KEY first using:\n\n`os.environ['GROQ_API_KEY']='YOUR_KEY'`")
    st.stop()

client = Groq(api_key=api_key)

# ----------------------------
# CREATE RETAIL DATASET
# ----------------------------
@st.cache_data
def create_dataset():
    data = {
        "ProductName": [
            "Wireless Mouse", "Gaming Keyboard", "USB-C Charger", "Bluetooth Speaker", "LED Monitor",
            "Smartwatch", "Noise Cancelling Headphones", "External Hard Drive", "Laptop Stand", "Webcam",
            "Office Chair", "Desk Lamp", "Portable Fan", "Power Bank", "HDMI Cable",
            "Phone Tripod", "Smart Light Bulb", "Wireless Earbuds", "Mechanical Keyboard", "Microphone",
            "Notebook", "Pen Set", "Coffee Mug", "Planner", "Table Clock",
            "Wall Calendar", "Candle - Lavender", "Candle - Vanilla", "Candle - Rose", "Candle - Sandalwood"
        ],
        "Category": [
            "Electronics", "Electronics", "Electronics", "Electronics", "Electronics",
            "Electronics", "Electronics", "Electronics", "Accessories", "Electronics",
            "Furniture", "Home Decor", "Home Appliances", "Electronics", "Accessories",
            "Accessories", "Home Decor", "Electronics", "Electronics", "Electronics",
            "Stationery", "Stationery", "Kitchenware", "Stationery", "Home Decor",
            "Home Decor", "Home Decor", "Home Decor", "Home Decor", "Home Decor"
        ],
        "Price": [
            1500, 3500, 1800, 2500, 22000,
            12000, 18000, 9000, 2500, 4000,
            15000, 3000, 2000, 3000, 800,
            1000, 1500, 8000, 7000, 4500,
            200, 500, 700, 1000, 1500,
            1000, 1200, 1300, 1100, 1400
        ],
        "Stock": [
            50, 40, 100, 35, 25,
            60, 30, 80, 55, 70,
            20, 45, 60, 90, 100,
            65, 85, 40, 50, 60,
            200, 150, 80, 100, 70,
            60, 30, 40, 35, 25
        ]
    }
    return pd.DataFrame(data)

data = create_dataset()

# ----------------------------
# FUNCTION TO HANDLE CHAT
# ----------------------------
def query_bot(prompt, df):
    try:
        system_prompt = f"""
        You are a helpful retail store assistant chatbot.
        You have access to the following product inventory:

        {df.to_string(index=False)}

        Your tasks:
        - If user asks for a product price, check if it exists in data and return the price.
        - If user asks what products are available, list some.
        - If user asks general retail/store-related questions (offers, return policy, categories, etc.), respond helpfully.
        - If product is not found, politely inform the user and suggest related products.
        """

        completion = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7
        )

        return completion.choices[0].message.content.strip()

    except Exception as e:
        return f"⚠ Error calling Groq API: {e}"

# ----------------------------
# STREAMLIT UI
# ----------------------------
st.set_page_config(
    page_title="🛒 Coles Assistant",
    layout="centered",
    initial_sidebar_state="collapsed"
)


st.markdown("""
<style>
    /* Main background with animated gradient */
    .stApp {
        background: linear-gradient(-45deg, #ee7752, #e73c7e, #23a6d5, #23d5ab);
        background-size: 400% 400%;
        animation: gradientShift 15s ease infinite;
    }

    @keyframes gradientShift {
        0% { background-position: 0% 50%; }
        50% { background-position: 100% 50%; }
        100% { background-position: 0% 50%; }
    }

    /* Hide default Streamlit elements */
    #MainMenu {visibility: hidden;}
    footer {visibility: hidden;}
    header {visibility: hidden;}

    /* Chat container with glassmorphism */
    .main .block-container {
        background: rgba(255, 255, 255, 0.1);
        backdrop-filter: blur(10px);
        border-radius: 30px;
        padding: 2rem;
        border: 1px solid rgba(255, 255, 255, 0.2);
        box-shadow: 0 8px 32px 0 rgba(31, 38, 135, 0.37);
        max-width: 900px;
        margin-top: 2rem;
    }

    /* Title styling */
    h1 {
        color: white !important;
        text-align: center;
        font-size: 3rem !important;
        font-weight: 800 !important;
        text-shadow: 2px 2px 4px rgba(0,0,0,0.3);
        margin-bottom: 0.5rem !important;
        animation: fadeInDown 0.8s ease-out;
    }

    @keyframes fadeInDown {
        from {
            opacity: 0;
            transform: translateY(-20px);
        }
        to {
            opacity: 1;
            transform: translateY(0);
        }
    }

    /* Subtitle */
    .stMarkdown p {
        color: rgba(255, 255, 255, 0.9) !important;
        text-align: center;
        font-size: 1.1rem;
        margin-bottom: 2rem;
    }

    /* Chat messages */
    .stChatMessage {
        background: rgba(255, 255, 255, 0.15) !important;
        backdrop-filter: blur(10px) !important;
        border-radius: 20px !important;
        border: 1px solid rgba(255, 255, 255, 0.2) !important;
        padding: 1rem !important;
        margin: 0.5rem 0 !important;
        animation: slideIn 0.3s ease-out;
        box-shadow: 0 4px 15px rgba(0, 0, 0, 0.1);
    }

    @keyframes slideIn {
        from {
            opacity: 0;
            transform: translateY(10px);
        }
        to {
            opacity: 1;
            transform: translateY(0);
        }
    }

    /* User message styling */
    [data-testid="stChatMessageContent"] {
        color: white !important;
        font-size: 1rem;
    }

    /* Chat input */
    .stChatInput {
        background: rgba(255, 255, 255, 0.2) !important;
        backdrop-filter: blur(10px) !important;
        border-radius: 25px !important;
        border: 2px solid rgba(255, 255, 255, 0.3) !important;
        padding: 0.5rem !important;
    }

    .stChatInput input {
        background: transparent !important;
        color: white !important;
        border: none !important;
        font-size: 1rem !important;
    }

    .stChatInput input::placeholder {
        color: rgba(255, 255, 255, 0.6) !important;
    }

    /* Spinner */
    .stSpinner > div {
        border-top-color: white !important;
    }

    /* Expander */
    .streamlit-expanderHeader {
        background: rgba(255, 255, 255, 0.2) !important;
        backdrop-filter: blur(10px) !important;
        border-radius: 15px !important;
        color: white !important;
        font-weight: 600 !important;
        border: 1px solid rgba(255, 255, 255, 0.2) !important;
    }

    .streamlit-expanderContent {
        background: rgba(255, 255, 255, 0.15) !important;
        backdrop-filter: blur(10px) !important;
        border-radius: 0 0 15px 15px !important;
        border: 1px solid rgba(255, 255, 255, 0.2) !important;
    }

    /* Dataframe styling */
    .dataframe {
        background: rgba(255, 255, 255, 0.9) !important;
        border-radius: 10px !important;
    }

    /* Avatar icons */
    .stChatMessage [data-testid="chatAvatarIcon"] {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
        box-shadow: 0 4px 15px rgba(0, 0, 0, 0.2);
    }

    /* Scrollbar */
    ::-webkit-scrollbar {
        width: 10px;
    }

    ::-webkit-scrollbar-track {
        background: rgba(255, 255, 255, 0.1);
        border-radius: 10px;
    }

    ::-webkit-scrollbar-thumb {
        background: rgba(255, 255, 255, 0.3);
        border-radius: 10px;
    }

    ::-webkit-scrollbar-thumb:hover {
        background: rgba(255, 255, 255, 0.5);
    }
</style>
""", unsafe_allow_html=True)

# ----------------------------
# STREAMLIT UI
# ----------------------------
st.title("🛍️ Coles Assistant")
st.markdown("✨ Ask about product prices, stock, or general store information")

# Initialize session state for conversation
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display previous messages
for msg in st.session_state.messages:
    with st.chat_message(msg["role"]):
        st.markdown(msg["content"])

# Chat input box
if user_input := st.chat_input("💬 Type your question here..."):
    # Save user message
    st.session_state.messages.append({"role": "user", "content": user_input})
    with st.chat_message("user"):
        st.markdown(user_input)

    # Generate assistant reply
    with st.chat_message("assistant"):
        with st.spinner("✨ Thinking..."):
            response = query_bot(user_input, data)
            st.markdown(response)

    # Save assistant reply
    st.session_state.messages.append({"role": "assistant", "content": response})

# Display inventory (optional)
#with st.expander("📦 View Product Inventory"):
 #   st.dataframe(data, use_container_width=True)


Overwriting app.py


In [ ]:
!ngrok config add-authtoken "my token here"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [75]:
!pip install pyngrok streamlit groq

In [ ]:
!ngrok config add-authtoken "my token here"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [77]:
!pkill ngrok


In [78]:
from pyngrok import ngrok
!streamlit run app.py &>/dev/null &

public_url = ngrok.connect(8501)
print("🌐 Your Coles AI (Groq) Chatbot is live at:", public_url)


🌐 Your Coles AI (Groq) Chatbot is live at: NgrokTunnel: "https://warren-unconforming-lexicographically.ngrok-free.dev" -> "http://localhost:8501"
